In [143]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2 as cv2
from PIL import Image
import os
import re
import json
from tqdm import tqdm

In [112]:
path = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/cropped/"
path_json = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/learning/output"
files = os.listdir(path)
files = [f for f in files if os.path.isfile(os.path.join(path, f))]

files_json = os.listdir(path_json)
files_json = [f for f in files_json if os.path.isfile(os.path.join(path_json, f))]

In [141]:
results = []

for i in tqdm(range(len(files)), desc="Processing files"):
    file = os.path.join(path, files[i])
    file_json = os.path.join(path_json, files_json[i])
    image = Image.open(file) 
    image_width, image_height = image.size 
    with open(file_json, "r") as f:
        data = json.load(f)
    # the coordation of the centers of department
    centers = []
    for i, box in enumerate(data["pred_boxes"]):
        x_center = int((box[0] + box[2]) / 2)
        y_center = int((box[1] + box[3]) / 2)

        # Normalize the image size
        x_normalized = (x_center / image_width) * 10
        y_normalized = (y_center / image_height) * 10
        centers.append({"id": i, "center": (x_normalized, y_normalized)})

    # make graph
    G = nx.Graph()
    # Add nodes to graph
    for center in centers:
        G.add_node(center["id"], pos=center["center"])
    # Add edges to graph 
    distance_threshold = 5
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            dist = np.linalg.norm(np.array(centers[i]["center"]) - np.array(centers[j]["center"]))
            if dist < distance_threshold:
                G.add_edge(centers[i]["id"], centers[j]["id"], weight=dist)

    pos = nx.get_node_attributes(G, "pos")
    shortest_length_path = nx.average_shortest_path_length(G)
    match = re.search(r"cropped_(\d{4})_(\d{4})", file)
    if match:
        year = match.group(1)  # Extracts the year (e.g., "2002")
        code = match.group(2)  # Extracts the code (e.g., "7003")
    else:
        year = None
        code = None

    results.append({"code": code, "year": year , "shortest_path_length": shortest_length_path})


Processing files: 100%|██████████| 101/101 [00:01<00:00, 95.36it/s]


In [146]:
data = pd.DataFrame(results)
data

,code,year,shortest_path_length
0,7003,2002,1.285965
1,7011,2002,1.315088
2,7014,2002,1.412982
3,7018,2002,1.301053
4,7102,2002,2.681053
...,...,...,...
96,7313,2010,1.613333
97,7314,2010,2.956842
98,7399,2010,1.832281
99,7404,2010,1.845965
